<a href="https://colab.research.google.com/github/monclai/assistente_estudantil/blob/main/Chatbot_escolar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação do google generative ai

In [ ]:
!pip install -q -U google-generativeai

Importação do google generative ai
Importação do userdata do google colab para uso da secret key

In [ ]:
import google.generativeai as genai
from google.colab import userdata

Importação de bibliotecas para auxiliar na exibição do texto markdown

In [ ]:
import textwrap
from IPython.display import display
from IPython.display import Markdown

Instanciação da secret key

In [ ]:
GOOGLE_API_KEY=userdata.get('api-key')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
class AssistenteEstudantil:
  def __init__(self):
    # Inicia as configurações do modelo
    self.__generation_config = {
      "temperature": 1,
      "top_p": 0.95,
      "top_k": 0,
      "max_output_tokens": 8192,
    }

    # Inicia as configurações de segurança
    self.__safety_settings = [
      {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
      },
      {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
      },
      {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
      },
      {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
      },
    ]

    self.__modelo = "gemini-1.5-pro-latest"
    self.__disciplina = ""
    self.__pergunta: str = ""

  def __gera_modelo(self, system_instruction = None):
    model = genai.GenerativeModel(model_name=self.__modelo,
                          generation_config=self.__generation_config,
                          system_instruction=system_instruction,
                          safety_settings=self.__safety_settings)

    return model

  def __defini_area_conhecimento(self):
    model = self.__gera_modelo()

    prompt_parts = [
      "Qual a área de conhecimento do ensino fundamental ou médio, pertence essa pegunta?",
      "pergunta: quais são as 3 leis de newton",
      "física",
      "pergunta: o que foi a reforma protestante",
      "historia",
      "pergunta: o que é revolta de canudos?",
      "história",
      "pergunta: quais são os relevos do brasil",
      "geografia",
      "pergunta: me explica cadeia alimentar",
      "biologia",
    ]

    prompt_parts.append(f"pergunta: {self.__pergunta}")
    prompt_parts.append(" ")
    prompt_parts

    response = model.generate_content(prompt_parts)
    self.__disciplina = response.text.rstrip(" \n")

  def __chat_conhecimento(self):
    system_instruction = f"aja como como um doutor em {self.__disciplina} com especialização em ensino de jovens que dar aula no ensino fundamental e busca ter uma didática simples, voltada para para o jovem, com linguagem a mesma linguagem utilizada em mídias sociais e direta"

    try:
      model = self.__gera_modelo(system_instruction)

      conversa = model.start_chat(history=[])

      conversa.send_message(self.__pergunta)
      # return(conversa.last.text)
      return(conversa)
    except Exception as e:
      print(f"Erro ao gerar resposta: {e}")
      resposta = genai.Conversa()
      resposta.send_message("Desculpe, tive um problema ao processar sua pergunta.")
      return resposta

  def __to_markdown(self, text):
    # text = text.replace('•', '  *')
    text = text.replace('•', '  ')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

  def __exibir_resposta(self, resposta):
    for message in resposta.history:
      if(message.role == "user"):
        message.role = "estudante"
      else:
        message.role = "Gemini"
      display(self.__to_markdown(f'**{message.role}**: {message.parts[0].text}'))

  def assistente_estudantil(self, pergunta: str):
    if not pergunta.strip():
      raise ValueError("Pergunta não pode estar vazia.")

    self.__pergunta = pergunta
    self.__defini_area_conhecimento()
    resposta = self.__chat_conhecimento()
    self.__exibir_resposta(resposta)

    return resposta


In [ ]:
gemini_assistente_estudantil = AssistenteEstudantil()

In [ ]:
resposta = gemini_assistente_estudantil.assistente_estudantil("O que é libertade poética")

> **estudante**: O que é libertade poética

> **Gemini**: ## Liberdade Poética: A Licença Para Dar um "Upgrade" na Realidade! 😜
> 
> Já leu um poema ou letra de música que parece meio... fora da caixa? Tipo, com rimas inusitadas, palavras inventadas ou até mesmo quebrando as regras da gramática? 🤯  Isso é a **liberdade poética** entrando em ação! 
> 
> É como um superpoder que os artistas usam para **brincar com a linguagem** e expressar suas ideias de um jeito único e criativo. 😎  Eles podem:
> 
> * **Inventar palavras novas:** Já ouviu falar em "borboletear" ou "aveleira"? São exemplos de palavras criadas por poetas para dar um toque especial à sua obra. 🦋
> * **Mudar a ordem das palavras:**  Quem disse que a frase precisa ser certinha? A liberdade poética deixa a gente brincar com a ordem das palavras para criar ritmo, sonoridade e impactar o leitor. 🎤
> * **Forçar a barra na rima:** Nem sempre a rima perfeita aparece, né? Mas com a liberdade poética, vale usar palavras com sons parecidos, mesmo que não sejam rimas perfeitas. 🎶
> * **Dar um tempo na gramática:** Às vezes, para dar ênfase ou criar um efeito poético, a liberdade poética deixa a gente quebrar algumas regras gramaticais, como concordância ou regência. 😜
> 
> **Lembre-se:** A liberdade poética não é bagunça! É uma ferramenta poderosa para tornar a arte mais expressiva e emocionante. 😉
> 
> **#FicaaDica:** Quer ver a liberdade poética em ação? Dá uma olhada nas letras de música dos seus artistas favoritos ou nos poemas modernistas! 🎤 📖 
